# Только по полю 'name', + Word2Vec + Кластеризация

In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import re
from sklearn.model_selection import train_test_split

In [2]:
%matplotlib notebook
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [3]:
df_train = pd.read_csv('train.csv', sep='\t')#, index_col='id')

In [4]:
#df_train = df_train.sample(1000, random_state=123)

In [5]:
from tqdm import tqdm

def ClearTag(TextData): #очиста от тэгов в столбце
    texts_without_tags = []
    
    for t in tqdm(TextData):
        texts_without_tags.append(BeautifulSoup(t, 'lxml').text)
    
    return texts_without_tags

In [6]:
#df_train['name'] = df_train['name'] + ' ' + df_train['description']
df_train = df_train.drop(['description'], axis=1)

In [7]:
df_train['name'] = ClearTag(df_train['name'])

100%|██████████| 200000/200000 [01:22<00:00, 2416.92it/s]


In [8]:
import pymorphy2 # Морфологический анализатор.
from collections import Counter # Не считать же частоты самим.
#import math # Корень квадратный.

morph=pymorphy2.MorphAnalyzer()

In [9]:
from nltk.corpus import stopwords # Импортируем стоп-слова русскго языка
Stop_Words = set(stopwords.words('russian'))

def getMeaningfullWords(text):
    words=[]
    tokens=re.findall('[А-Яа-яЁёa-zA-Z]+\-[А-Яа-яЁёa-zA-Z]+|[А-Яа-яЁёa-zA-Z]+', text)
    tokens_eng=re.findall('[a-zA-Z]+\-[a-zA-Z]+|[a-zA-Z]+', text)
    
    for t in tokens:
        if not t in Stop_Words:
            if t in tokens_eng:
                words.append(t)
            else:
                pv=morph.parse(t)
                for p in pv:
                    if p.tag.POS in ['ADJF', 'NOUN', 'VERB']:
                        if not p.normal_form in Stop_Words:
                            words.append(p.normal_form)
                        break
                
#    tokens_eng=re.findall('[a-zA-Z]+\-[a-zA-Z]+|[a-zA-Z]+', text)
#    for t in tokens_eng:
#        words.append(t)
    
    return (words)

In [ ]:
getMeaningfullWords("Account подкрался manger не заметно. Красивая ты вошла в мою грешную жизнь")

In [10]:
#num = df_train['name'].size #считываем размер выборки
#num = 10

clean_train_words = [] #собираем "мешок слов"

for i in tqdm(df_train.id):
#    print('i = ', i, ' - ', df_train['name'][i])
    clean_train_words.append(getMeaningfullWords(df_train['name'][i]))

100%|██████████| 200000/200000 [04:37<00:00, 721.86it/s] 


In [ ]:
#clean_train_words

### Word2Vec

In [ ]:
#clean_train_words = Load_Obj('clean_train_words(name).pcl')


In [11]:
%%time

from gensim.models.word2vec import Word2Vec 

# список параметров, которые можно менять по вашему желанию
num_features = 600  # итоговая размерность вектора каждого слова
min_word_count = 5  # минимальная частотность слова, чтобы оно попало в модель
num_workers = 4     # количество ядер вашего процессора, чтоб запустить обучение в несколько потоков
context = 10        # размер окна 
downsampling = 1e-3 # внутренняя метрика модели

model = Word2Vec(clean_train_words, workers=num_workers, size=num_features,
                 min_count=min_word_count, window=context, sample=downsampling)

C:\Users\Andrei\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


Wall time: 9.61 s


In [ ]:
model.save('Model_Work2Vec_train.mdl')

In [ ]:
model.wv.most_similar('менеджер')

In [12]:
import numpy as np

def text_to_vec(words, model, size):
    text_vec = np.zeros((size,), dtype="float32")
    n_words = 0

    index2word_set = set(model.wv.index2word)
    for word in words:
        if word in index2word_set:
            n_words = n_words + 1
            text_vec = np.add(text_vec, model.wv[word])
    
    if n_words != 0:
        text_vec /= n_words
    return text_vec

In [13]:
def texts_to_vecs(texts, model, size):
    texts_vecs = np.zeros((len(texts), size), dtype="float32")
    
    for i, text in enumerate(texts):
        texts_vecs[i] = text_to_vec(text, model, size)

    return texts_vecs

In [14]:
%%time
train_vecs = texts_to_vecs(clean_train_words, model, num_features)

Wall time: 1min 49s


In [ ]:
#Save_Obj(train_vecs, 'train_vecs.pcl')

In [ ]:
train_vecs.shape

In [15]:
import pandas as pd

In [16]:
Y = df_train['target'].values
#Y = pd.DataFrame(Y)

In [17]:
import umap

In [18]:
umap_emb = umap.UMAP(n_components=3, metric='cosine')

In [20]:
%%time
show_reduced = umap_emb.fit_transform(train_vecs, y=Y)

C:\Users\Andrei\Anaconda3\lib\site-packages\umap\spectral.py:229: UserWarning: Embedding 979 connected components using meta-embedding (experimental)
  n_components


Wall time: 3h 17min 44s


In [ ]:
#show_reduced.embedding_

In [ ]:
#%%time
#from sklearn.decomposition import PCA
#pca=PCA(n_components=3)
#show_reduced_PCA = pca.fit_transform(train_vecs, y=Y)
#show_reduced = show_reduced_PCA

In [ ]:
#Save_Obj(show_reduced, 'show_reduced(train_name).pcl')

In [ ]:
show_reduced_PCA = show_reduced.embedding_

In [ ]:
fig = plt.figure(1, figsize=(8, 6))
ax = Axes3D(fig, elev=60, azim=150)

ax.scatter(show_reduced_PCA[:, 0], show_reduced_PCA[:, 1], show_reduced_PCA[:, 2], c=df_train.target,
           cmap=plt.cm.Set1,  s=5)
plt.show()


### учим модель

In [25]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, accuracy_score, classification_report
from sklearn.metrics import roc_auc_score, roc_curve

In [26]:
Y = df_train['target']


In [23]:
#del df_train

In [27]:
X_train, X_test, y_train, y_test = train_test_split(show_reduced, Y, test_size=0.33, random_state=123)

In [28]:
print('X_train ', X_train.shape)
print('X_test ',X_test.shape)
print('y_train ', y_train.shape)
print('y_test ',y_test.shape)

X_train  (134000, 3)
X_test  (66000, 3)
y_train  (134000,)
y_test  (66000,)


In [29]:
%%time
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier


#model_DTC = DecisionTreeClassifier(max_depth = 25, random_state=123)
model_RFC = RandomForestClassifier(n_estimators = 100, n_jobs = -1) 

C:\Users\Andrei\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


Wall time: 59.1 ms


In [30]:
%%time
model_RFC.fit( X_train, y_train )
#model_DTC.fit( X_train, y_train )

Wall time: 13.4 s


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [31]:
predict = model_RFC.predict(X_test)
#predict = model_DTC.predict(X_test)


In [32]:
print(classification_report(y_test, predict))

             precision    recall  f1-score   support

          0       0.99      0.99      0.99     35162
          1       0.99      0.99      0.99     30838

avg / total       0.99      0.99      0.99     66000



             precision    recall  f1-score   support

          0       0.84      0.95      0.89        39
          1       0.91      0.74      0.82        27

avg / total       0.87      0.86      0.86        66

0.8447293447293447

In [33]:
roc_auc_score(y_test, predict)

0.9918356951764594

## Обучение на полном df_test

In [35]:
#Уже обучена = грузить из файла
# 1. загрузить модель из файла RFC_df_train.pcl
# 2
#
#
model_RFC.fit(show_reduced, Y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [25]:
df_test = pd.read_csv('test.csv', sep='\t', index_col='id')
#df_test = df_test.sample(1000, random_state=123)

In [ ]:
#df_test.info()

In [26]:
#df_test['name'] = df_test['name'] + ' ' + df_test['description']
df_test = df_test.drop(['description'], axis=1)

In [ ]:
#model.save('Model_Work2Vec_train.mdl')

In [27]:
df_test['name'] = ClearTag(df_test['name'])

100%|██████████| 170179/170179 [01:16<00:00, 2211.29it/s]


In [28]:
#num = df_test.size

clean_test_words = [] #собираем "мешок слов"
count = 0
for i in tqdm(df_test.index):
    clean_test_words.append(getMeaningfullWords(df_test['name'][i]))

100%|██████████| 170179/170179 [04:25<00:00, 640.83it/s]


In [ ]:
Save_Obj(clean_test_words, 'clean_test_words(name).tcl')

In [ ]:
#save_umap(umap_emb)

In [29]:
#model = Word2Vec(clean_test_words, workers=num_workers, size=num_features,
#                 min_count=min_word_count, window=context, sample=downsampling)
model.train(clean_test_words, total_examples=len(clean_test_words), epochs=10)

(3505743, 5407620)

In [30]:
%%time
test_vecs = texts_to_vecs(clean_test_words, model, num_features)

Wall time: 1min 37s


In [ ]:
#Save_Obj(test_vecs, 'test_vecs.tcl')

In [ ]:
test_vecs.shape

In [31]:
%%time
show_reduced_test = umap_emb.transform(test_vecs)

Wall time: 29min 35s


In [ ]:
show_reduced_test

In [ ]:
fig2 = plt.figure(2, figsize=(8, 6))
ax2 = Axes3D(fig2, elev=60, azim=150)

ax2.scatter(show_reduced_test[:, 0], show_reduced_test[:, 1], show_reduced_test[:, 2], #c=df_train.target,
           cmap=plt.cm.Set1,  s=5)
plt.show()

In [32]:
%%time
predict_test = model_RFC.predict(show_reduced_test)

Wall time: 6.14 s


In [34]:
output = pd.DataFrame(data={'id':df_test.index, 'target':predict_test} )

In [35]:
output[output['target']==1].count(), output[output['target']==0].count()

(id        80740
 target    80740
 dtype: int64, id        89439
 target    89439
 dtype: int64)

In [ ]:
df_test['target'] = predict_test

In [ ]:
#df_test

In [ ]:
df_train[df_train['name'] == 'Продавец-консультант'].count()

In [ ]:
import string as str
s = 'оборудования'
for string in df_train['name'][:1000]:
    if s.find(string) >=0:
        print(string)

In [36]:
output.to_csv("RFC_UMAP_Word2Vec_name.csv", sep=',', encoding='utf-8',index=False )

 <b color='red'>НЕЗАПУСКАТЬ!!!</b>

In [ ]:
df_train = pd.read_csv('train.csv', sep='\t', index_col='id')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_train.drop('target', axis=1), df_train['target'], 
                                                    test_size=0.30, random_state=123)

X_test['target'] = y_test

In [ ]:
X_test.to_csv('train1900.csv', sep='\t', index_label='id')

In [ ]:
#SAVE model RandomForestClissifyer
import _pickle as cPickle
with open('RFC_df_train.pcl', 'wb') as f:
    cPickle.dump(model_RFC, f)


In [ ]:
import _pickle as cPickle
with open('RFC_df_train.pcl', 'rb') as f:
    model_RFC = cPickle.load(f)

In [ ]:
#LOAD clean_train_words from the file
filename = 'clean_train_words(name).pcl'
#filename = 'clean_train_words(name-descr).csv'
#filename = 'clean_test_words(name-descr).csv'

import _pickle as cPickle
with open(filename, 'rb') as f:
    clean_train_words = cPickle.load(f)
    #clean_test_words = cPickle.load(f)

In [ ]:
#SAVE clean_..._words from the file
import _pickle as cPickle


filename = 'clean_train_words(name).pcl'
clean_words = clean_train_words

#filename = 'clean_test_words(name).csv'
#clean_words = clean_test_words


with open(filename, 'wb') as f:
    cPickle.dump(clean_words, f)

In [ ]:
#SAVE umap_emb the file
import _pickle as cPickle


filename = 'umap_emb(name).pcl'

#filename = 'clean_test_words(name).csv'
#clean_words = clean_test_words


with open(filename, 'wb') as f:
    cPickle.dump(umap_emb, f)



In [ ]:
#SAVE show_reduced the file
import _pickle as cPickle


filename = 'show_reduced(name).pcl'
data = show_reduced

#filename = 'clean_test_words(name).csv'
#clean_words = clean_test_words


with open(filename, 'wb') as f:
    cPickle.dump(data, f)

In [ ]:
import _pickle as cPickle
def Save_Obj(data, filename):
    with open(filename, 'wb') as f:
        cPickle.dump(data, f)
    
def Load_Obj(filename):
    with open(filename, 'rb') as f:
        data = cPickle.load(f)
    return (data)    

In [ ]:
import pickle

def save_umap(umap):
    for attr in ["_tree_init", "_search", "_random_init"]:
        if hasattr(umap, attr):
            delattr(umap, attr)
    return pickle.dumps(umap, pickle.HIGHEST_PROTOCOL)

def load_umap(s):
    umap = pickle.loads(s)
    from umap.nndescent import make_initialisations, make_initialized_nnd_search
    umap._random_init, umap._tree_init = make_initialisations(
        umap._distance_func, umap._dist_args
    )
    umap._search = make_initialized_nnd_search(
        umap._distance_func, umap._dist_args
    )
    return umap